### Student Information
Name: 蕭靖澂 Ching-Cheng Hsiao

Student ID: 110033632

GitHub ID: hsiaooo

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

In [1]:
### Begin Assignment Here

### Load kaggle dataset

In [2]:
# load the libraries
import os
import json
import pandas as pd

# load the json
path     = os.path.dirname(os.path.abspath("tweets_DM.json"))
# raw_data = pd.read_json(path+"/dataset/tweets_DM.json",lines=True)
# tweets   = pd.json_normalize(data=raw_data['_source'])
# identify = pd.read_csv(basePath+"/dataset/data_identification.csv")
# emotion  = pd.read_csv(basePath+"/dataset/emotion.csv")

In [3]:
# tweets.head()

In [4]:
# # rename column names
# tweets = tweets.rename(index=str, columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags"})

# # add identify tags to dataframe
# tweets = pd.merge(tweets, identify, on="tweet_id")

# tweets.head()

In [5]:
# # get training set and test set
# train_df = tweets[tweets["identification"] == "train"]
# test_df  = tweets[tweets["identification"] == "test"]

# # drop identification tags
# train_df.drop(columns=["identification"], inplace=True)
# test_df.drop(columns=["identification"], inplace=True)

# print(f'Training set\n{train_df.head()}\n')
# print(f'Testing set\n{test_df.head()}')

In [6]:
# add emotion column
# train_df = pd.merge(train_df, emotion, on="tweet_id")
# test_df = test_df.assign(emotion="")

In [7]:
# use tweet_id as index
# train_df.set_index("tweet_id",inplace=True)
# test_df.set_index("tweet_id",inplace=True)

# train_df.head()

In [8]:
# save to pickle file
# train_df.to_pickle("train_df.pkl")
# test_df.to_pickle("test_df.pkl")

In [9]:
# load a pickle file
train_df = pd.read_pickle(path+"/train_df.pkl")
test_df = pd.read_pickle(path+"/test_df.pkl")

### Pre-processing

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(preserve_case=False)
tfidf = TfidfVectorizer(max_features=20000, stop_words='english', tokenizer=tknzr.tokenize)

# fitting
tfidf.fit(train_df['text'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=20000, stop_words='english',
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f59915b3f40>>)

In [11]:
# transforming training sets
X_train = tfidf.transform(train_df['text'])
X_train.shape

(1455563, 20000)

In [12]:
# transforming testing sets
X_test = tfidf.transform(test_df['text'])
X_test.shape

(411972, 20000)

In [13]:
# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [14]:
# data_dum = pd.get_dummies(y_train.iloc[:])
# y_train  = pd.DataFrame(data_dum)

#### XGBoost Regressor

In [ ]:
import xgboost as xgb

emotion = pd.Categorical(y_train.iloc[:], categories=y_train.iloc[:].unique(), ordered=True)

y_train.iloc[:] = emotion.codes

xgbr = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgbr.fit(X_train, y_train)